# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
keras = tf.keras
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler
import plotly.graph_objects as go
import plotly.offline as pyo
from math import sqrt
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Importing Data

In [2]:
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

# Data Pre-Procesing

In [3]:
DAYS_TO_PREDICT = int(input('Enter Number Of Days You Want to Predict : '))  # last days to predict  

recovered_df = recovered_df.groupby("Country/Region").sum().reset_index()
Country_recovered =  recovered_df['Country/Region'].values
recovered_df = recovered_df.drop(['Lat','Long','Country/Region'], axis=1)
recovered_df = recovered_df[recovered_df.columns].T
recovered_df.columns = Country_recovered
compare_recovered_df=recovered_df.iloc[-DAYS_TO_PREDICT-1:,:]                 # last days to predict
recovered_df= recovered_df.iloc[:-DAYS_TO_PREDICT,:]
                                
Date_index = recovered_df.index
recovered_df = recovered_df.diff().fillna(0)
compare_recovered_df = compare_recovered_df.diff().dropna(0)
recovered_df = recovered_df.abs()
recovered_df

Enter Number Of Days You Want to Predict : 5


,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,Azerbaijan,Bahamas,Bahrain,Bangladesh,Barbados,Belarus,Belgium,Belize,Benin,Bhutan,Bolivia,Bosnia and Herzegovina,Botswana,Brazil,Brunei,Bulgaria,Burkina Faso,Burma,Burundi,Cabo Verde,Cambodia,Cameroon,Canada,Central African Republic,Chad,Chile,China,Colombia,Comoros,Congo (Brazzaville),...,Senegal,Serbia,Seychelles,Sierra Leone,Singapore,Slovakia,Slovenia,Somalia,South Africa,South Sudan,Spain,Sri Lanka,Sudan,Suriname,Sweden,Switzerland,Syria,Taiwan*,Tajikistan,Tanzania,Thailand,Timor-Leste,Togo,Trinidad and Tobago,Tunisia,Turkey,US,Uganda,Ukraine,United Arab Emirates,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
1/22/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/23/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/24/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/25/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1/26/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6/18/20,1502.0,9.0,135.0,1.0,0.0,0.0,209.0,746.0,1.0,2.0,117.0,2.0,511.0,1975.0,0.0,1288.0,40.0,0.0,1.0,1.0,0.0,22.0,0.0,13534.0,0.0,61.0,1.0,2.0,0.0,0.0,0.0,0.0,502.0,15.0,12.0,4510.0,5.0,961.0,2.0,0.0,...,110.0,72.0,0.0,3.0,774.0,6.0,0.0,39.0,589.0,28.0,0.0,24.0,0.0,26.0,0.0,0.0,0.0,0.0,62.0,0.0,1.0,0.0,0.0,0.0,2.0,1382.0,6924.0,66.0,516.0,704.0,9.0,4.0,35.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0
6/19/20,302.0,28.0,118.0,0.0,2.0,0.0,1130.0,706.0,3.0,40.0,133.0,0.0,591.0,2781.0,0.0,1252.0,27.0,1.0,9.0,3.0,668.0,22.0,0.0,17051.0,0.0,67.0,0.0,5.0,0.0,0.0,0.0,1978.0,536.0,0.0,9.0,5050.0,0.0,1365.0,0.0,0.0,...,72.0,53.0,0.0,22.0,747.0,4.0,0.0,0.0,2905.0,5.0,0.0,25.0,120.0,0.0,0.0,0.0,0.0,0.0,68.0,0.0,11.0,0.0,8.0,0.0,8.0,1494.0,7600.0,6.0,587.0,755.0,6.0,0.0,107.0,0.0,1.0,22.0,0.0,15.0,50.0,0.0
6/20/20,330.0,12.0,128.0,0.0,0.0,0.0,355.0,588.0,15.0,34.0,191.0,0.0,503.0,1048.0,0.0,1474.0,20.0,0.0,0.0,2.0,416.0,0.0,0.0,25148.0,0.0,19.0,0.0,4.0,0.0,0.0,0.0,154.0,508.0,3.0,4.0,5118.0,19.0,1309.0,30.0,0.0,...,71.0,67.0,0.0,14.0,765.0,0.0,0.0,27.0,2501.0,0.0,0.0,26.0,239.0,0.0,0.0,0.0,5.0,0.0,64.0,0.0,10.0,0.0,5.0,0.0,3.0,1312.0,10745.0,0.0,386.0,758.0,0.0,1.0,17.0,0.0,1.0,0.0,0.0,40.0,0.0,0.0
6/21/20,472.0,8.0,98.0,0.0,11.0,0.0

# Enter countries you want to forecast

In [4]:
i = int(input('Number of countries you want to consider : '))
countries = [input('Enter country You Want to forecast : ') for i in range(i)]

Number of countries you want to consider : 3
Enter country You Want to forecast : Germany
Enter country You Want to forecast : Italy
Enter country You Want to forecast : China


In [5]:
countries

['Germany', 'Italy', 'China']

In [6]:
recovered_df = recovered_df.loc[:,countries]   
compare_recovered_df = compare_recovered_df.loc[:,countries]  # last days to predict 

In [7]:
recovered_df

,Germany,Italy,China
1/22/20,0.0,0.0,0.0
1/23/20,0.0,0.0,2.0
1/24/20,0.0,0.0,6.0
1/25/20,0.0,0.0,3.0
1/26/20,0.0,0.0,10.0
...,...,...,...
6/18/20,248.0,1089.0,5.0
6/19/20,125.0,1363.0,0.0
6/20/20,637.0,546.0,19.0
6/21/20,131.0,440.0,1.0


# Scalling the data

In [8]:
sc = RobustScaler()
recovered_df_scaled = sc.fit_transform(recovered_df)
recovered_df_scaled = pd.DataFrame(recovered_df_scaled)
recovered_df_scaled

,0,1,2
0,-0.271875,-0.502669,-0.147493
1,-0.271875,-0.502669,-0.144543
2,-0.271875,-0.502669,-0.138643
3,-0.271875,-0.502669,-0.143068
4,-0.271875,-0.502669,-0.132743
...,...,...,...
148,-0.116875,0.025716,-0.140118
149,-0.193750,0.158661,-0.147493
150,0.126250,-0.237749,-0.119469
151,-0.190000,-0.289180,-0.146018


# Enter the number of days you want to predict

In [9]:
# DAYS_TO_PREDICT = int(input('Enter Number Of Days You Want to Predict : '))
predict_recovered_index = pd.date_range(start=Date_index[-1],periods=DAYS_TO_PREDICT + 1,closed='right')
predict_recovered_index = predict_recovered_index.strftime('%Y/%m/%d')

# Creating class for building model

In [10]:
class Corona_recovered_model():
    def create_dataset(self, dataset, time_steps=1, column = 0):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_steps-1):
          a = dataset[i:(i+time_steps), column]
          dataX.append(a)
          dataY.append(dataset[i + time_steps, column])
        return np.array(dataX), np.array(dataY)

    def Model(self,dframe,column):
        df = dframe.values
        df = df.astype('float32')
        
        time_steps = 1
        trainX, trainY = self.create_dataset(df, time_steps, column)
        trainX = np.reshape(trainX, (trainX.shape[0],1,trainX.shape[1]))
        
        
        keras.backend.clear_session()
        tf.random.set_seed(42)
        np.random.seed(42)
        recover_model = keras.models.Sequential([
            keras.layers.InputLayer(input_shape=[None, 1]),
            keras.layers.Conv1D(filters=16, kernel_size=2,
                      strides=1, padding="causal",
                      activation="relu",
                      ),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.LSTM(30,kernel_initializer = 'he_normal',
                              return_sequences = True),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(15,kernel_initializer = 'he_normal',
                                             activation = 'relu'),
            keras.layers.Dense(1,activation = 'linear',kernel_initializer = 'he_normal'),
            keras.layers.Lambda(lambda x: x * 200)
             ])
        
        optimizer = keras.optimizers.Adadelta(lr = 0.001)
        
        recover_model.compile(loss=['mae'],optimizer=optimizer)   
                
        recover_model.fit(trainX, trainY, epochs=100,
                          batch_size = 10, verbose = 0)

        recover_test_seq  = trainX[-1:]
        recover_test_seq = recover_test_seq
        recover_test_seq = recover_test_seq.reshape(len(recover_test_seq), time_steps, 1)
        recover_preds = []
        
        for _ in range (DAYS_TO_PREDICT):
            recover_pred = recover_model.predict(recover_test_seq)
            recover_preds.append(recover_pred)
            recover_new_seq = recover_pred
            recover_test_seq = recover_new_seq
            recover_test_seq = recover_test_seq.reshape(len(recover_test_seq ), time_steps, 1)

        

        return recover_preds

# Calling the class and predicting the cases for every individual country

In [11]:
NN = Corona_recovered_model()
recovered_predictions = []
for i in range(len(countries)):
    result = NN.Model(recovered_df_scaled,i) 
    recovered_predictions.append(result)
recovered_predictions

[[array([[[-0.0126717]]], dtype=float32),
  array([[[-0.16891699]]], dtype=float32),
  array([[[-0.27531242]]], dtype=float32),
  array([[[-0.271673]]], dtype=float32),
  array([[[-0.2716276]]], dtype=float32)],
 [array([[[-0.38521892]]], dtype=float32),
  array([[[-0.44007283]]], dtype=float32),
  array([[[-0.46694353]]], dtype=float32),
  array([[[-0.4815018]]], dtype=float32),
  array([[[-0.48978478]]], dtype=float32)],
 [array([[[-0.12932706]]], dtype=float32),
  array([[[-0.13518564]]], dtype=float32),
  array([[[-0.13867575]]], dtype=float32),
  array([[[-0.1407583]]], dtype=float32),
  array([[[-0.14200227]]], dtype=float32)]]

In [12]:
recovered_predictions = (np.array(recovered_predictions).transpose())
recovered_predictions = recovered_predictions.reshape(((DAYS_TO_PREDICT),len(countries)))
recovered_predictions = sc.inverse_transform(recovered_predictions)
recovered_predictions = pd.DataFrame(recovered_predictions)
recovered_predictions = recovered_predictions.abs()
recovered_predictions['Dates'] = predict_recovered_index
recovered_predictions.set_index('Dates', inplace = True)
recovered_predictions.columns = countries
recovered_predictions.head()

,Germany,Italy,China
Dates,,,
2020/06/23,414.725281,242.063782,12.316254
2020/06/24,164.732819,129.009888,8.344131
2020/06/25,5.499878,73.629395,5.977844
2020/06/26,0.323212,43.624817,4.565872
2020/06/27,0.395844,26.553589,3.722458


# Visualizing The Recovered Cases For A Country As Per The User's Choice

### List of Infected Countries

In [13]:
Infected_Countries = pd.DataFrame(countries)
Infected_Countries.columns = ['Countries']
pd.set_option("max_rows", None)
Infected_Countries

,Countries
0,Germany
1,Italy
2,China


### Select The Country

In [14]:
Country = input('Enter the name of the country from above list you would like to see the daily cases till now : ')

Enter the name of the country from above list you would like to see the daily cases till now : Italy


## Till Date Daily Cases of Recovery from coronavirus

In [15]:
fig = go.Figure()
fig.update_layout(template='plotly_dark',title={'text': 'Recovered Daily Cases','y':0.9,'x':0.5,'xanchor': 'center',
                                                'yanchor': 'top'},
                  xaxis_title="Dates",
                  yaxis_title="Number Of Cases")
fig.add_trace(go.Scatter(x=recovered_df.index, 
                         y=recovered_df[Country],
                         mode='lines+markers',
                         line=dict(color='green', width=2)))

## Predictions of recovered cases from coronavirus

In [16]:
fig = go.Figure()
fig.update_layout(template='plotly_dark',title={'text': 'Recovered Cases Predictions','y':0.9,'x':0.5,'xanchor': 'center',
                                                'yanchor': 'top'},xaxis_title="Dates",yaxis_title="Number Of Cases")
fig.update_xaxes(tickangle=90, showticklabels = True, type = 'category')
fig.add_trace(go.Scatter(x=recovered_predictions.index, 
                         y=recovered_predictions[Country],
                         mode='lines+markers',
                         line=dict(color='white', width=2)))

# Root mean square error between last days predictions(Only for comment last days to predict)

In [17]:
compare_recovered_df

,Germany,Italy,China
6/23/20,682.0,1159.0,8.0
6/24/20,597.0,1526.0,7.0
6/25/20,342.0,614.0,10.0
6/26/20,385.0,890.0,8.0
6/27/20,369.0,969.0,11.0


In [18]:
recovered_predictions

,Germany,Italy,China
Dates,,,
2020/06/23,414.725281,242.063782,12.316254
2020/06/24,164.732819,129.009888,8.344131
2020/06/25,5.499878,73.629395,5.977844
2020/06/26,0.323212,43.624817,4.565872
2020/06/27,0.395844,26.553589,3.722458


In [19]:
for country in countries:
  realVals = compare_recovered_df[country]
  predictedVals = recovered_predictions[country]
  rmse = sqrt(mean_squared_error(realVals , predictedVals))
  print('RMSE for {}'.format(country) , rmse)

RMSE for Germany 362.04095886360676
RMSE for Italy 968.391447909962
RMSE for China 4.502673096704999
